In [37]:
from layout import Layout

## Definition
The (logical) product is defined as $A \otimes B := (A, \tilde A \circ B)$.

In terms of the pseudo code:
```python
size = self.size() * other.cosize()
compl = self.complement(size)
compo = compl.composite(other)
return Layout.from_concate(self, compo)
```

## Yet to be Done

In [41]:
# Layout.from_string('(2,2):(4,1)').visualize().logical_product(Layout.from_string('6:1')).visualize()
# Layout.from_string('(2,2):(4,1)').visualize().logical_product(Layout.from_string('(4,2):(2,1)').visualize()).visualize()
# Layout.from_string('(2,5):(5,1)').visualize().logical_product(Layout.from_string('(3, 4):(5, 6)').visualize(), by_mode=True).visualize()
# Layout.from_string('(2,5):(5,1)').visualize().blocked_product(Layout.from_string('(3, 4):(1, 3)').visualize()).visualize()